## Import Python Libraries

In [3]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score

from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score

## Read in Data and Inital Exploration

In [4]:
org_data = pd.read_csv('./adult.csv', header=0)
org_data.shape

(32561, 15)

In [5]:
org_data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [6]:
org_data.nunique()

age                  73
workclass             9
fnlwgt            21648
education            16
education.num        16
marital.status        7
occupation           15
relationship          6
race                  5
sex                   2
capital.gain        119
capital.loss         92
hours.per.week       94
native.country       42
income                2
dtype: int64

In [7]:
org_data['occupation'].value_counts()

Prof-specialty       4140
Craft-repair         4099
Exec-managerial      4066
Adm-clerical         3770
Sales                3650
Other-service        3295
Machine-op-inspct    2002
?                    1843
Transport-moving     1597
Handlers-cleaners    1370
Farming-fishing       994
Tech-support          928
Protective-serv       649
Priv-house-serv       149
Armed-Forces            9
Name: occupation, dtype: int64

Inital Notes:
 - No null values but '?' instead which will need to be converted
 - education.num appears to just be a ordinal numeric encoding of the eduction feature, can probably drop one of the columns
 - from the description of fnlwgt feature provided, it appears that similar fnlwgt values should indicate simliar socio-economic characteristics but because this is      based on state by state samples this only holds where comparing across the same states. This may cause issues ??
 - Different categorical features are going to require different encodings, native. country should probably be encoded using some form of geographical location to represetn distance but in this case Onehotencoding will do. 

In [8]:
# replace ? with nan

In [9]:
org_data[org_data == "?"] = np.nan

In [10]:
org_data.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

Data has missing values in 'workclass', 'occupation', 'native.country' all of which appear to be categorical data 

In [11]:
missing_columns = ['workclass','occupation','native.country']

In [12]:
org_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  31978 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [13]:
org_data.describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [14]:
#remove education and keep education number and remove fnlweight as we don't know where it comes from

org_data = org_data.drop(['education','fnlwgt'], axis=1)

## Mean Imputation

In [15]:
mean_data = org_data.copy()

In [16]:
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
mean_imputer.fit(mean_data[missing_columns])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [17]:
mean_data[missing_columns] = mean_imputer.transform(mean_data[missing_columns])

In [18]:
mean_data.isnull().sum()

age               0
workclass         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [19]:
mean_data

,age,workclass,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Private,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,Private,10,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [20]:
M_cat_cols = ['marital.status','relationship','income','sex','race']

In [21]:
M_OH_data = pd.get_dummies(mean_data[M_cat_cols], drop_first=True) # prevent dummy variable trap
mean_data.drop(M_cat_cols, axis=1, inplace=True)
for col in missing_columns:
    le = LabelEncoder()
    labeled_data = le.fit_transform(mean_data[col]) 
    mean_data[col] = labeled_data
mean_data = pd.concat([mean_data,M_OH_data], axis=1)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(mean_data)
mean_data = pd.DataFrame(scaled_data, columns=mean_data.columns)

In [22]:
mean_data

,age,workclass,education.num,occupation,capital.gain,capital.loss,hours.per.week,native.country,marital.status_Married-AF-spouse,marital.status_Married-civ-spouse,...,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,income_>50K,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,1.000000,0.428571,0.533333,0.692308,0.0,1.000000,0.397959,0.95,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.890411,0.428571,0.533333,0.230769,0.0,1.000000,0.173469,0.95,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.671233,0.428571,0.600000,0.692308,0.0,1.000000,0.397959,0.95,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.506849,0.428571,0.200000,0.461538,0.0,0.895317,0.397959,0.95,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.328767,0.428571,0.600000,0.692308,0.0,0.895317,0.397959,0.95,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.068493,0.428571,0.600000,0.769231,0.0,0.000000,0.397959,0.95,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
32557,0.136986,0.428571,0.733333,0.923077,0.0,0.000000,0.377551,0.95,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
32558,0.315068,0.428571,0.533333,0.461538,0.0,0.000000,0.397959,0.95,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
32559,0.561644,0.428571,0.533333,0.000000,0.0,0.000000,0.397959,0.95,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Cold Deck Imputation

The simplest way of implementing Cold Deck imputation would be to use sklears knn imputer with k=1 where you just select the corresponding value of the most similar row.


In [23]:
CD_data = org_data.copy()

In [24]:
missing_data = CD_data[missing_columns]
full_data = CD_data.drop(missing_columns, axis=1)

Convert all categorical columns that are not missing values to OneHotEncoded data (None of the remain categorical variables have )

In [25]:
cat_cols = ['marital.status', 'relationship','income','sex','race']

In [26]:
OH_data = pd.get_dummies(full_data[cat_cols], drop_first=True) # prevent dummy variable trap
full_data.drop(cat_cols, axis=1, inplace=True)
full_data = pd.concat([full_data,OH_data], axis=1)

#scale all columns not containing missing data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(full_data)
full_data = pd.DataFrame(scaled_data, columns=full_data.columns)

In [27]:
#encode columns with missing values 
missing_data.fillna("0", inplace=True)
for col in missing_columns:

    #encode columns with missing data
    le = LabelEncoder()
    labeled_data = le.fit_transform(missing_data[col])
    missing_data[col] = labeled_data

    #scale columns with missing data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(missing_data[[col]])
    missing_data[col] = scaled_data
    missing_data[col] = missing_data[col].apply(lambda x: np.nan if x==0 else x)

In [28]:
CD_data = pd.concat([missing_data, full_data],axis=1)
CD_data

,workclass,occupation,native.country,age,education.num,capital.gain,capital.loss,hours.per.week,marital.status_Married-AF-spouse,marital.status_Married-civ-spouse,...,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,income_>50K,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,NaN,NaN,0.95122,1.000000,0.533333,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.5,0.285714,0.95122,0.890411,0.533333,0.0,1.000000,0.173469,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,NaN,0.95122,0.671233,0.600000,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.5,0.500000,0.95122,0.506849,0.200000,0.0,0.895317,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.5,0.714286,0.95122,0.328767,0.600000,0.0,0.895317,0.397959,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.5,0.785714,0.95122,0.068493,0.600000,0.0,0.000000,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
32557,0.5,0.928571,0.95122,0.136986,0.733333,0.0,0.000000,0.377551,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
32558,0.5,0.500000,0.95122,0.315068,0.533333,0.0,0.000000,0.397959,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
32559,0.5,0.071429,0.95122,0.561644,0.533333,0.0,0.000000,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
CD_imp = KNNImputer(n_neighbors=1)

In [30]:
CD_data = pd.DataFrame(CD_imp.fit_transform(CD_data), columns = CD_data.columns)

In [31]:
CD_data

,workclass,occupation,native.country,age,education.num,capital.gain,capital.loss,hours.per.week,marital.status_Married-AF-spouse,marital.status_Married-civ-spouse,...,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,income_>50K,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,0.5,0.285714,0.95122,1.000000,0.533333,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.5,0.285714,0.95122,0.890411,0.533333,0.0,1.000000,0.173469,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.5,0.285714,0.95122,0.671233,0.600000,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.5,0.500000,0.95122,0.506849,0.200000,0.0,0.895317,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.5,0.714286,0.95122,0.328767,0.600000,0.0,0.895317,0.397959,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.5,0.785714,0.95122,0.068493,0.600000,0.0,0.000000,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
32557,0.5,0.928571,0.95122,0.136986,0.733333,0.0,0.000000,0.377551,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
32558,0.5,0.500000,0.95122,0.315068,0.533333,0.0,0.000000,0.397959,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
32559,0.5,0.071429,0.95122,0.561644,0.533333,0.0,0.000000,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Regression Imputation

In [32]:
reg_data = org_data.copy()

To keep regression more simple we will only use complete features as predictor variables rather than using random imputation first etc.

In [33]:
missing_features = reg_data[missing_columns]
pred_features = reg_data.drop(missing_columns, axis=1)

In [34]:
cat_cols = ['marital.status', 'relationship','income','sex','race']
OH_data = pd.get_dummies(pred_features[cat_cols], drop_first=True) # prevent dummy variable trap
pred_features.drop(cat_cols, axis=1, inplace=True)
pred_features = pd.concat([pred_features,OH_data], axis=1)

#scale all columns not containing missing data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(pred_features)
pred_features = pd.DataFrame(scaled_data, columns=pred_features.columns)

#encode columns with missing values 
missing_features.fillna("0", inplace=True)
for col in missing_columns:

    #encode columns with missing data
    le = LabelEncoder()
    labeled_data = le.fit_transform(missing_features[col])
    missing_features[col] = labeled_data

    missing_features[col] = missing_features[col].apply(lambda x: np.nan if x==0 else x)

In [35]:
reg_data = pd.concat([missing_features, pred_features],axis=1)

In [36]:
test_data = reg_data[reg_data.isna().any(axis=1)]
train_data = reg_data.dropna()

S_test_data = test_data.copy()
S_train_data = train_data.copy()

In [37]:
test_data

,workclass,occupation,native.country,age,education.num,capital.gain,capital.loss,hours.per.week,marital.status_Married-AF-spouse,marital.status_Married-civ-spouse,...,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,income_>50K,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,NaN,NaN,39.0,1.000000,0.533333,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,NaN,39.0,0.671233,0.600000,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,4.0,3.0,NaN,0.328767,0.600000,0.0,0.689624,0.602041,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
14,NaN,NaN,39.0,0.465753,1.000000,0.0,0.648301,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
18,4.0,6.0,NaN,0.068493,0.733333,0.0,0.648301,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32533,NaN,NaN,39.0,0.246575,0.800000,0.0,0.000000,0.551020,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
32534,NaN,NaN,39.0,0.178082,0.800000,0.0,0.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
32541,NaN,NaN,39.0,0.739726,1.000000,0.0,0.000000,0.091837,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
32543,NaN,NaN,39.0,0.328767,0.533333,0.0,0.000000,0.316327,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [38]:
for feature in missing_features.columns:
    y_train = train_data[feature]
    x_train = train_data.drop(missing_columns,axis=1)

    x_test = test_data.drop(missing_columns,axis=1)


    log_reg = LogisticRegression(solver='newton-cg',random_state=0).fit(x_train, y_train)
    out_prob = log_reg.predict_proba(x_test)
    test_data[feature] = log_reg.predict(x_test)



In [39]:

for feature in missing_features.columns:
    y_train = S_train_data[feature]
    x_train = S_train_data.drop(missing_columns,axis=1)

    x_test = S_test_data.drop(missing_columns,axis=1)


    log_reg = LogisticRegression(solver='newton-cg',random_state=0).fit(x_train, y_train)
    
    probs =log_reg.predict_proba(x_test) # get the list of probabilites of each class for each example to use as a sampling distribution
    classes_range = range(1,train_data[feature].nunique()+1) 
    S_test_data[feature] = np.apply_along_axis(lambda x: np.random.choice(a=classes_range, size = 1, p=x), 1, probs) #for each row sample for the predicted probabilites and assign the class

In [42]:
test_data

,workclass,occupation,native.country,age,education.num,capital.gain,capital.loss,hours.per.week,marital.status_Married-AF-spouse,marital.status_Married-civ-spouse,...,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,income_>50K,sex_Male,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,4.0,4.0,39.0,1.000000,0.533333,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4.0,1.0,39.0,0.671233,0.600000,0.0,1.000000,0.397959,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,4.0,4.0,39.0,0.328767,0.600000,0.0,0.689624,0.602041,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
14,4.0,10.0,39.0,0.465753,1.000000,0.0,0.648301,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
18,4.0,10.0,39.0,0.068493,0.733333,0.0,0.648301,0.397959,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32533,4.0,10.0,39.0,0.246575,0.800000,0.0,0.000000,0.551020,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
32534,4.0,4.0,39.0,0.178082,0.800000,0.0,0.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
32541,4.0,10.0,39.0,0.739726,1.000000,0.0,0.000000,0.091837,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
32543,4.0,8.0,39.0,0.328767,0.533333,0.0,0.000000,0.316327,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [43]:
reg_data = pd.concat([train_data,test_data])

In [44]:
S_reg_data = pd.concat([train_data, S_test_data])

## Test/Train Split 

Create a dictionary of datasets for each imputaion method

In [45]:
imputation_sets = [mean_data, CD_data, reg_data, S_reg_data]
name_sets = ['M', 'CD', 'DR', 'SR']

In [46]:
cleaned_data = {}
for imp in zip(imputation_sets,name_sets):
    X, y = imp[0][imp[0].columns[imp[0].columns != 'income_>50K']], imp[0]['income_>50K']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    cleaned_data[imp[1]] = [X_train, X_test, y_train, y_test]

## Decision Tree

In [47]:
cv = StratifiedKFold(2, shuffle=True, random_state=42)

In [48]:
for dataset in name_sets:
    tree = DecisionTreeClassifier()
    tree.fit(cleaned_data[dataset][0], cleaned_data[dataset][2])
    score, perm_scores, pvalue= permutation_test_score(tree, cleaned_data[dataset][1],cleaned_data[dataset][3], cv=cv, n_permutations=30)
    print('{}|{}|{}|{}|{}'.format(dataset,'Accuracy',round(score,4), 'P-value',round(pvalue,4)))

M|Accuracy|0.8009|P-value|0.0323
CD|Accuracy|0.8002|P-value|0.0323
DR|Accuracy|0.8025|P-value|0.0323
SR|Accuracy|0.8021|P-value|0.0323


## Random Forest

In [49]:
for dataset in name_sets:
    forest = RandomForestClassifier()
    forest.fit(cleaned_data[dataset][0], cleaned_data[dataset][2])
    score, perm_scores, pvalue= permutation_test_score(forest, cleaned_data[dataset][1],cleaned_data[dataset][3], cv=cv, n_permutations=30)
    print('{}|{}|{}|{}'.format('Accuracy',round(score,4), 'P-value',round(pvalue,4)))

Accuracy|0.8482|P-value|0.0323
Accuracy|0.8477|P-value|0.0323
Accuracy|0.8406|P-value|0.0323
Accuracy|0.8409|P-value|0.0323


## Naive Bayes: Gaussian

In [51]:
for dataset in name_sets:
    NBG = GaussianNB()
    NBG.fit(cleaned_data[dataset][0], cleaned_data[dataset][2])
    score, perm_scores, pvalue= permutation_test_score(NBG, cleaned_data[dataset][1],cleaned_data[dataset][3], cv=cv, n_permutations=30)
    print('{}|{}|{}|{}'.format('Accuracy',round(score,4), 'P-value',round(pvalue,4)))

Accuracy|0.6771|P-value|0.1613
Accuracy|0.6771|P-value|0.1613
Accuracy|0.688|P-value|0.1935
Accuracy|0.6885|P-value|0.2258


## Naive Bayes: Multinomial

In [52]:
for dataset in name_sets:
    NBM = MultinomialNB()
    NBM.fit(cleaned_data[dataset][0], cleaned_data[dataset][2])
    score, perm_scores, pvalue= permutation_test_score(NBM, cleaned_data[dataset][1],cleaned_data[dataset][3], cv=cv, n_permutations=30)
    print('{}|{}|{}|{}'.format('Accuracy',round(score,4), 'P-value',round(pvalue,4)))

Accuracy|0.7199|P-value|1.0
Accuracy|0.7198|P-value|1.0
Accuracy|0.7577|P-value|1.0
Accuracy|0.7536|P-value|1.0


## Baysian Networks

Because of the computational difficulty of finding baysian network structures I have only implemented it for a single imputation method. Even so running predictions can only be perfomered on small sub samples and so accuracy can only be determined from this sampling.

A full implementation for all datasets would simply involved putting the proceeding code in a loop as in the above classifers. To make the code runnable this has not been done as terminataion time could be very long and uncheckable depending on systems. 

In [57]:
full_data = pd.concat([cleaned_data['M'][0],cleaned_data['M'][2]],axis=1)

Use Hillclimbmethod to create a network structure

In [59]:
scoring_method = K2Score(data=full_data)
est = HillClimbSearch(data=full_data)
estimated_model = est.estimate(scoring_method=scoring_method, max_indegree=4, max_iter=int(1e3))

7%|▋         | 68/1000 [00:25<05:49,  2.67it/s]


In [61]:
estimated_model.edges()

OutEdgeView([('age', 'marital.status_Never-married'), ('age', 'income_>50K'), ('education.num', 'occupation'), ('occupation', 'workclass'), ('hours.per.week', 'relationship_Own-child'), ('native.country', 'race_Asian-Pac-Islander'), ('native.country', 'education.num'), ('native.country', 'race_Black'), ('marital.status_Married-AF-spouse', 'relationship_Wife'), ('marital.status_Married-AF-spouse', 'income_>50K'), ('marital.status_Married-civ-spouse', 'relationship_Not-in-family'), ('marital.status_Married-civ-spouse', 'income_>50K'), ('marital.status_Married-civ-spouse', 'sex_Male'), ('marital.status_Married-civ-spouse', 'relationship_Wife'), ('marital.status_Married-civ-spouse', 'capital.gain'), ('marital.status_Married-civ-spouse', 'marital.status_Married-spouse-absent'), ('marital.status_Married-civ-spouse', 'race_Black'), ('marital.status_Married-civ-spouse', 'marital.status_Married-AF-spouse'), ('marital.status_Married-spouse-absent', 'race_Other'), ('marital.status_Never-married',

In [62]:
model = BayesianNetwork(estimated_model.edges())
model.fit(full_data)

In [65]:
#single batch of 200 as larger batching fails
sample = cleaned_data['M'][1].sample(150)
y_pred = model.predict(sample)


100%|██████████| 150/150 [00:25<00:00,  5.94it/s]


In [68]:
#single sample accuracy
accuracy_score(cleaned_data['M'][3][sample.index],y_pred)

0.8666666666666667

### Comments on Classifers

Based on the results above it is clear to see that the decision tree based classifiers perform better over all with decision trees averaging around 80% accuracy and Random Forrest around 84%. Both also had significant P-values of 0.03 which is significant. The trade off with both being they were much slower to run than the naive bayes classifers, in particular Random forrest was notciable slower than all the other classifers bar Baysian Network (Discussed below).

The gaussian and multinomial Naive Bayes preform noticably poorer. With Gaussian getting around 68% accruacy and P-value of 0.3 and multinomial getting a accuracy of 74% with a P-value of 1.(This P-value would appear to be a mistake and is probably due to some internal workings of the "Permutation_test_score" function that needs further exploration).

The Baysian Network was the most difficult classifers to work with and the slowest due to the need to build the network structed and fit parameters to the model. Due to this is and the limitations of my own hardware I was unable to get more than one data set to run using the Baysian network and even then I was only able to predict samples of at most 200 rows at a time.(These samples still took minutes to run and often still failed). Due to this issue I decided not to run more tests on the other datasets. On the small samples run it did appear to have the highest accruacy at 86% but because of the small sample size this is very inconclusive and should be disregarded.

Based on this data I think the strongest classifers is the Random Forrest as it provides the best accruacy, the model has a low value and while it is slower than the others it is not so slow as to be unusable like the Baysian Network.

### Comments on Imputation Methods

The quality and improvement the imputation had seemed to depend on the classifer being used. The type of impuation used had little impact on the performance of the decsion tree classifers with accuracy and P-values being approximatley the same for all types of imputation. 

There was noticable improvment when using the regression imputations on the accuracy of the predictors from approx. 72% to 76% in the case of the multinomial and 68% to 69% for the Gaussian. This did however also increase the size of the P-value for the Guassian from around 0.16 to 0.19 for detemenistic and 0.22 for stochastic imputation.(P-value for multinomial was 1 for all impuations). 

The accuracy of the Baysian Network was only tested on the mean imputed data but seemed to perform well on it. (again sample size is small so should be disregarded.)

For this data set as there is actaully rather small number of missing values and those that do have missing values are mostly categorical. I would recommend using mean imputation (Most frequent for categorical) as it is the fastest and easiest to achieve if using the tree classifers. That being said if having to use a Baysian classifer then using a detemenistic logistic regressor does appear to provide noticable improvement on the performance of the classifer while being relativly easy to compute. 
